In [16]:
# データセット分割
from sklearn.model_selection import train_test_split
# 標準化
from sklearn.preprocessing import StandardScaler
# 正答率
from sklearn.metrics import accuracy_score

# K-近傍法
from sklearn.neighbors import KNeighborsClassifier
# サポートベクターマシン
from sklearn.svm import SVC
# 決定木
from sklearn.tree import DecisionTreeClassifier
# ランダムフォレスト、Adaブースト
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
# ナイーブベイズ
from sklearn.naive_bayes import GaussianNB
# 線形判別分析
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# 二次判別分析
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from mlxtend.plotting import plot_decision_regions

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys

import itertools
import talib as ta # テクニカル指標

from df_method import rise_fall_rate, moving_average, GCDC, df_shift, add_data, RSI, Z_score_normalization, Min_Max_normalization

## 変数宣言

In [17]:
csv_path = './csv_realtime/'
x_days_later = 1
start_date = '2009-3-10' # 2009-3-10:リーマン・ショック後, 2013:自民党政権発足
end_date = '2018-10-31' # 米中間選挙前
isDebug = False

## 分類器と対応する名前の用意

In [18]:
names = [
    "Nearest Neighbors",
    "Linear SVM",
    "RBF SVM",
    "Decision Tree",
    "Random Forest",
    "AdaBoost",
    "Naive Bays",
    "Linear Discriminant Analysis",
    "Quadratic Discriminant Analysis"
]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    DecisionTreeClassifier(max_depth=5, random_state=20181224),
    RandomForestClassifier(max_depth=5, random_state=20181224),
    AdaBoostClassifier(),
    GaussianNB(),
    LinearDiscriminantAnalysis(),
    QuadraticDiscriminantAnalysis()
]

## 学習データと対応する名前の用意

In [19]:
# 13個はいける、？
df_name = [
    'USD_JPY',
    'EUR_JPY',
    'EUR_USD',
    'nikkei',
    'TOPIX',
    'DOW30',
    'NASDAQ',
    'treasury10',
    'SMA',
    'BB',
    'RSI',
]

df_list = []

# OHLC
df_list.append(add_data(csv_path+'USD_JPY.csv', x_days_later)) # 標準で盛り込む
df_list.append(add_data(csv_path+'EUR_JPY.csv', x_days_later))
df_list.append(add_data(csv_path+'EUR_USD.csv', x_days_later))
df_list.append(add_data(csv_path+'nikkei.csv', x_days_later))
df_list.append(add_data(csv_path+'TOPIX.csv', x_days_later))
df_list.append(add_data(csv_path+'DOW30.csv', x_days_later))
df_list.append(add_data(csv_path+'NASDAQ.csv', x_days_later))

# 米国債
df = pd.read_csv(csv_path+'treasury_10.csv', index_col='Date', parse_dates=True)
df_list.append(rise_fall_rate(df, x_days_later)) 

# テクニカル指標を求めるために使用-----------------------------------------
df = pd.read_csv(csv_path+'USD_JPY.csv', index_col=0, parse_dates=True)
df_close = df['USD_JPY_Close']

# 指数移動平均----------------------------------------------------
sma5 = ta.SMA(df_close, timeperiod=5)
sma25 = ta.SMA(df_close, timeperiod=25)
sma75 = ta.SMA(df_close, timeperiod=75)
sma5.name = 'sma5'
sma25.name = 'sma25'
sma75.name = 'sma75'
SMA = pd.DataFrame(sma5)
SMA = SMA.join(sma25)
SMA = SMA.join(sma75)
SMA = SMA.dropna()
# 5日移動平均
"""
target = df_close[SMA.index[0]:]
df = SMA['sma5'] - target
df.name = 'sma5'
df_list.append(df)
"""
SMA_diff = sma5 - sma25
SMA_diff = SMA_diff.dropna()
SMA_diff.name = 'SMA_5_25'
#rint(SMA_diff)
df_list.append(SMA_diff)



# ★ボリンジャーバンド-----------------------------------------------------
upper1, middle1, lower1 = ta.BBANDS(df_close, timeperiod=5, nbdevup=1, nbdevdn=1)
upper2, middle2, lower2 = ta.BBANDS(df_close, timeperiod=5, nbdevup=2, nbdevdn=2)
upper3, middle3, lower3 = ta.BBANDS(df_close, timeperiod=5, nbdevup=3, nbdevdn=3)
upper3.name = 'upper3'
upper2.name = 'upper2'
upper1.name = 'upper1'
middle1.name = 'middle'
lower1.name = 'lower1'
lower2.name = 'lower2'
lower3.name = 'lower3'
BB = pd.DataFrame(upper3)
BB = BB.join(upper2)
BB = BB.join(upper1)
BB = BB.join(middle1)
BB = BB.join(lower1)
BB = BB.join(lower2)
BB = BB.join(lower3)
BB = BB.dropna()
#df_list['BB'] = BB # そのまま入れる

# ラベルデータを作成する列
target = df_close[BB.index[0]:]
# 空のデータフレーム作成
#df_t = pd.DataFrame(index=BB.index, columns=['over3', 'over2', 'over1', 'over0', 'less0', 'less1', 'less2', 'less3'])
df_t = pd.DataFrame(index=BB.index, columns=['over3', 'over2', 'over1', 'neutral', 'less1', 'less2', 'less3'])
df_t = df_t.fillna(0) #０で埋める
# 条件にあった値を置換する
df_t.loc[BB["upper3"] <= target, 'over3'] = 1
df_t.loc[(BB["upper2"] <= target) & (target < BB["upper3"]), 'over2'] = 1
df_t.loc[(BB["upper1"] <= target) & (target < BB["upper2"]), 'over1'] = 1
#df_t.loc[(BB["middle"] <= target) & (target < BB["upper1"]), 'over0'] = 1
#df_t.loc[(BB["lower1"] <= target) & (target < BB["middle"]), 'less0'] = 1
df_t.loc[(BB["lower1"] <= target) & (target < BB["upper1"]), 'neutral'] = 1
df_t.loc[(BB["lower2"] <= target) & (target < BB["lower1"]), 'less1'] = 1
df_t.loc[(BB["lower3"] <= target) & (target < BB["lower2"]), 'less2'] = 1
df_t.loc[target < BB["lower3"], 'less3'] = 1
df_list.append(df_t)

# ★RSI
RSI = ta.RSI(df_close, timeperiod=14)
RSI.name = 'RSI'
#df_list['RSI'] = RSI # そのまま入れる

# ラベルデータを作成する列
target = RSI#df_x[target_name]
# 空のデータフレーム作成
df_t = pd.DataFrame(index=RSI.index, columns=['over70', 'neutral', 'less30'])
df_t = df_t.fillna(0) #０で埋める
# 条件にあった値を置換する
df_t.loc[70 <= target, 'over70'] = 1
df_t.loc[(30 < target) & (target < 70), 'neutral'] = 1
df_t.loc[target <= 30, 'less30'] = 1
df_list.append(df_t)

#print(df_x)

if len(df_name) != len(df_list):
    print('df_name =', len(df_name))
    print('df_list =', len(df_list))
    sys.stderr.write('Error occurred!')

In [20]:
# 移動平均
"""
sma5 = ta.SMA(df_close, timeperiod=5)
sma25 = ta.SMA(df_close, timeperiod=25)
sma75 = ta.SMA(df_close, timeperiod=75)
sma5.name = 'sma5'
sma25.name = 'sma25'
sma75.name = 'sma75'
SMA = pd.DataFrame(sma5)
SMA = SMA.join(sma25)
SMA = SMA.join(sma75)
SMA = SMA.dropna()
df_list.append(SMA)

# ゴールデンクロス、デッドクロス（GCDC）
GCDC5_25 = sma5 - sma25
GCDC25_75 = sma25 - sma75
GCDC5_25 = GCDC5_25.dropna()
GCDC25_75 = GCDC25_75.dropna()
df_list.append(GCDC5_25)
df_list.append(GCDC25_75)
"""

# ボリンジャーバンド
"""
upper, middle, lower = ta.BBANDS(df_close, timeperiod=5)
upper2, middle2, lower2 = ta.BBANDS(df_close, timeperiod=5, nbdevup=2, nbdevdn=2)
upper2.name = 'upper2'
upper.name = 'upper'
middle.name = 'middle'
lower.name = 'lower'
lower2.name = 'lower2'
BB = pd.DataFrame(upper2)
BB = BB.join(upper)
BB = BB.join(middle)
BB = BB.join(lower)
BB = BB.join(lower2)
BB = BB.dropna()
df_list.append(BB)
"""

"""
# ★MACD---------------------------------------------
MACD, MACD_signal, MACD_hist = ta.MACD(df_close, fastperiod=12, slowperiod=26, signalperiod=9)
#MACD.name = 'MACD'
print(MACD_hist)
#print(type(MACD))
#df_list.append(MACD)

# ★RSI
RSI = ta.RSI(df_close, timeperiod=14)
RSI.name = 'RSI'
df_list.append(RSI)

# ★ストキャスティクスRSI
SRSI, SRSI_d = ta.STOCHRSI(df_close, timeperiod=14, fastk_period=5, fastd_period=3)
#print(stockRSI)
"""

if len(df_name) != len(df_list):
    print('df_name =', len(df_name))
    print('df_list =', len(df_list))
    sys.stderr.write('Error occurred!')

## メイン

In [21]:
result_csv = pd.DataFrame(columns=names)
result_csv['combination'] = ''
#print(result_csv)
index_number = 0

for r in range(len(df_list)+1):
    if isDebug == True:
        print('r =', r)
    
    # 組み合わせのリストを作成
    combination_list = list(itertools.combinations(df_list, r))
    combination_name_list = list(itertools.combinations(df_name, r))
    
    # 組み合わせのリストから組み合わせのタプルを抽出
    for combination, i, combination_name in zip(combination_list, range(len(combination_list)), combination_name_list):
        
        # 学習データの作成--------------------------------------------------
        # ドル円の終値と始値の差
        df_x = pd.read_csv(csv_path+'USD_JPY.csv', index_col='Date', parse_dates=True)
        df_x = df_x.apply(np.log)*100
        df_x = df_x['USD_JPY_Close'] - df_x['USD_JPY_Open']
        df_x = df_x['1989-10-16':]
        df_x = df_x.rename('USD_JPY_diff')
        df_x = pd.DataFrame(df_x)
        #print(df_x)
        # 学習データの作成--------------------------------------------------
        
        # 組み合わせのタプルからデータフレームを一つにまとめる
        for df in combination:
            df_x = df_x.join(df, how='inner', rsuffix='_'+str(i))
        
        if isDebug == True:
            print(df_x)
            
        # ラベルデータの作成------------------------------------------------
        # 2010年以降のみ使用
        if end_date == '':
            df_x = df_x[start_date:]
        else:
            df_x = df_x[start_date:end_date]

        # ▲なぜか動く--どんどんアペンドされている！！
        target = df_x['USD_JPY_diff']
        #print(target)

        # ラベルデータ用のデータフレームを作成
        df_t = pd.DataFrame(index=df_x.index, columns=['t'])
        df_t = df_t.fillna(0)

        # 条件に合わせて0と1を代入
        df_t.loc[0 <= target, 't'] = 1
        df_t.loc[target < 0, 't'] = 0

        # スライドさせる●ここの処理は適切ではない！x_days_laterが1のときのみ有効
        df_t = df_t.shift(-1 * x_days_later, axis=0)
        df_t = df_t.drop(df_t.index[-1*x_days_later:], axis=0)
        df_x = df_x.drop(df_x.index[-1*x_days_later:], axis=0)

        # ???
        df_t = df_t['t']
        df_t = df_t.astype('int64')
        #print(df_t)
        # ラベルデータの作成--------------------------------------------------
            
        # numpy行列に変換
        x_data = df_x.values
        t_data = df_t.values

        # 学習データとラベルデータを訓練データとテストデータに分割する
        # x_train, x_test, t_train, t_test = train_test_split(x_data, t_data, test_size=0.1, random_state=None)
        n_train = int(len(df_x)*0.9)              # 訓練データ長
        #print(n_train)
        x_train,x_test = np.vsplit(np.array(x_data), [n_train]) # 学習データを訓練用とテスト用に分割
        t_train,t_test = np.split(np.array(t_data), [n_train]) # ラベルデータを訓練用とテスト用に分割

        # 標準化
        sc = StandardScaler()
        sc.fit(x_train)
        x_train_std = sc.transform(x_train)
        x_test_std = sc.transform(x_test)

        
        #print(r, df_x)
        
        result_csv.loc[index_number] = 0
        for name, model in zip(names, classifiers):
            model.fit(x_train_std, t_train) # 学習
            
            
            # 正答率を計算
            pred_train = model.predict(x_train_std)
            accuracy_train = accuracy_score(t_train, pred_train)
            pred_test = model.predict(x_test_std)
            accuracy_test = accuracy_score(t_test, pred_test)

            # 正答率を表示
            if isDebug == True:
                print(name)
                print('訓練データでの正答率: %.2f' % accuracy_train)
                print('テストデータでの正答率: %.2f' % accuracy_test)
                print("\n")
    
            # 入力次元が２でデバッグ中でないの場合のみグラフを表示
            if (len(df_x.columns) == 2 and isDebug == True):
                plt.style.use('ggplot')
    
                x_combined_std = np.vstack((x_train_std, x_test_std))
                y_combined = np.hstack((t_train, t_test))

                fig = plt.figure(figsize=(13, 8))
                plot_decision_regions(x_combined_std, y_combined, clf=model, res=0.02)
                plt.show()
            
    
            # ログを書き込む
            result_csv.at[index_number, 'combination'] = ' '.join(combination_name)
            result_csv.at[index_number, name] = accuracy_test
        
        result_csv.to_csv('./log_scikit_learn/log.csv', index=False)# 上書き
        #result_csv.tail().to_csv('./log_scikit_learn/log.csv', index=False, mode='a')# 追記
        index_number += 1

C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminan

C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:716: RuntimeWarning: divide by zero encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:716: RuntimeWarning: invalid value encountered in multiply
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:719: RuntimeWarning: divide by zero en

C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.wa

C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:716: RuntimeWarning: divide by zero encountered in power

C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.wa

...

C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.wa

C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.wa

C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.wa

C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.wa

C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.wa

## 各手法の最高正答率

In [24]:
result_csv = pd.read_csv('log_scikit_learn/log2.csv')

for name in names:
    if name == combination:
        continue
    print(name)
    print('最高正答率')
    # print(result_csv[].max())
    #print(result_csv[name].max())
    x = result_csv[result_csv[name] == result_csv[name].max()]
    print(x[name])
    print(x['combination'])
    print("\n")

Nearest Neighbors
最高正答率
94     0.611814
329    0.611814
Name: Nearest Neighbors, dtype: float64
94        USD_JPY TOPIX SMA
329    USD_JPY TOPIX SMA BB
Name: combination, dtype: object


Linear SVM
最高正答率
1406    0.572052
Name: Linear SVM, dtype: float64
1406    EUR_USD nikkei TOPIX DOW30 treasury10 SMA
Name: combination, dtype: object


RBF SVM
最高正答率
94    0.624473
Name: RBF SVM, dtype: float64
94    USD_JPY TOPIX SMA
Name: combination, dtype: object


Decision Tree
最高正答率
738     0.580786
1258    0.580786
1259    0.580786
1680    0.580786
1692    0.580786
1933    0.580786
2014    0.580786
Name: Decision Tree, dtype: float64
738                        USD_JPY TOPIX DOW30 NASDAQ SMA
1258                    USD_JPY TOPIX DOW30 NASDAQ SMA BB
1259                   USD_JPY TOPIX DOW30 NASDAQ SMA RSI
1680        USD_JPY nikkei TOPIX NASDAQ treasury10 BB RSI
1692                USD_JPY TOPIX DOW30 NASDAQ SMA BB RSI
1933    USD_JPY nikkei TOPIX NASDAQ treasury10 SMA BB RSI
2014    USD_JPY EUR_

## 各手法の平均正答率

In [23]:
for name in names:
    if name == combination:
        continue
    print(name)
    print('平均正答率')
    print(result_csv[name].mean())
    print("\n")

Nearest Neighbors
平均正答率
0.5155063700523196


Linear SVM
平均正答率
0.48816142859422335


RBF SVM
平均正答率
0.49036935831426814


Decision Tree
平均正答率
0.518662387365893


Random Forest
平均正答率
0.5097303075526708


AdaBoost
平均正答率
0.5140631944726901


Naive Bays
平均正答率
0.5088775651774567


Linear Discriminant Analysis
平均正答率
0.5021312284015864


Quadratic Discriminant Analysis
平均正答率
0.4936885202990126


